In [40]:
import sympy as sp
import numpy as np

def get_quantum_advantage(classical_runtime, quantum_runtime, hardware_slowdown, quantum_improvement_rate=2, year=2024):
    n = sp.Symbol('n')
    classical_runtime = sp.sympify(classical_runtime)
    quantum_runtime = sp.sympify(quantum_runtime)

    problem_size = sp.Eq(classical_runtime, quantum_runtime * (hardware_slowdown * (1/(quantum_improvement_rate**(year - 2024)))))
    solutions = sp.solve(problem_size, n)
    real_solutions = [solution.evalf() for solution in solutions if solution.is_real]

    if not real_solutions:
        raise ValueError("No real solutions found.")
    
    max_solution = max(real_solutions, key=lambda x: x.evalf())
    return max_solution.evalf()



def get_quantum_feasible(year, roadmap, physical_logical_qubits_ratio=1000):
    year = float(year)
    years = np.array(list(roadmap.keys()))
    qubits = np.array(list(roadmap.values()))

    if year in roadmap:
        number_of_physical_qubits = roadmap[year]
    elif year > max(years):
        regression = np.polyfit(years[years > 2024], qubits[years > 2024], 1)
        number_of_physical_qubits = np.polyval(regression, year)
    else:
        number_of_physical_qubits = np.interp(year, years, qubits)

    problem_size = 2 ** (number_of_physical_qubits / physical_logical_qubits_ratio)
    return problem_size



ibm_roadmap = {
    2021.0: 127,
    2022.0: 433,
    2023.0: 1121,
    2024.0: 1386,
    2025.0: 4158,
    2033.0: 100000

}

# Define the quantum feasible and quantum advantage functions as a function of the year
def qf(roadmap):
    return lambda year: float(sp.log(get_quantum_feasible(year, roadmap),10).evalf())

def qa(classical_runtime, quantum_runtime, hardware_slowdown,  quantum_improvement_rate):
    return lambda year: float(sp.log(get_quantum_advantage(classical_runtime, quantum_runtime, hardware_slowdown, quantum_improvement_rate, year=year),10).evalf())


# Create the functions
f = qf(ibm_roadmap)
a = qa('n^3', 'n^2', 10**6, 2)


def target_function(year):
    return f(year) - a(year)

def find_zero_bisection(year_low, year_high, tolerance=0.1):
    while (year_high - year_low) > tolerance:
        print(f"Year Low: {year_low}, Year High: {year_high}")
        year_mid = (year_high + year_low) / 2
        if target_function(year_mid) * target_function(year_low) <= 0:
            year_high = year_mid
        else:
            year_low = year_mid
    return (year_high + year_low) / 2

# Example usage:
# Replace year_low and year_high with your initial guesses
year_zero = find_zero_bisection(year_low=2024, year_high=2100)
print(f"Year where Quantum Feasible and Quantum Advantage are closest: {year_zero}")



Year Low: 2024, Year High: 2100
Year Low: 2024, Year High: 2062.0
Year Low: 2024, Year High: 2043.0
Year Low: 2024, Year High: 2033.5
Year Low: 2024, Year High: 2028.75
Year Low: 2024, Year High: 2026.375
Year Low: 2025.1875, Year High: 2026.375
Year Low: 2025.78125, Year High: 2026.375
Year Low: 2026.078125, Year High: 2026.375
Year Low: 2026.078125, Year High: 2026.2265625
Year where Quantum Feasible and Quantum Advantage are closest: 2026.115234375


In [71]:
4 - 0.117394172703787 * 0.5
(a(2024.5))

3.94130291364811

In [61]:
a = qa('n^2', 'sqrt(n)', hardware_slowdown=10**6,  quantum_improvement_rate=1.5**1)
for i in range(2024, 2050):
    hw = 10**6 * ((1.02**(i - 2024)))
    print(f"{i}: {a(i)} : {a(i) - a(i+1)}")



2024: 4.00000000000000 : 0.117394172703787
2025: 3.88260582729621 : 0.117394172703788
2026: 3.76521165459242 : 0.117394172703787
2027: 3.64781748188864 : 0.117394172703788
2028: 3.53042330918485 : 0.117394172703788
2029: 3.41302913648106 : 0.117394172703787
2030: 3.29563496377728 : 0.117394172703787
2031: 3.17824079107349 : 0.117394172703788
2032: 3.06084661836970 : 0.117394172703788
2033: 2.94345244566591 : 0.117394172703787
2034: 2.82605827296213 : 0.117394172703788
2035: 2.70866410025834 : 0.117394172703787
2036: 2.59126992755455 : 0.117394172703787
2037: 2.47387575485076 : 0.117394172703788
2038: 2.35648158214698 : 0.117394172703787
2039: 2.23908740944319 : 0.117394172703788
2040: 2.12169323673940 : 0.117394172703788
2041: 2.00429906403561 : 0.117394172703787
2042: 1.88690489133183 : 0.117394172703787
2043: 1.76951071862804 : 0.117394172703787
2044: 1.65211654592425 : 0.117394172703787
2045: 1.53472237322046 : 0.117394172703787
2046: 1.41732820051668 : 0.117394172703787
2047: 1.299